<a href="https://colab.research.google.com/github/Sally-Ama-Sampson/Estimating-Tair-from-CS-generated-Data-/blob/main/Morphometrics_elements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install momepy
!pip install osmnx

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import momepy as mm
import osmnx as ox
import numpy as np
from shapely.geometry import box
import utilsALL

In [ ]:
gdf = gpd.read_file('ALLNAIR.shp')
BNEWAOI= gdf.to_crs(32737)
BNEWAOI.crs

In [ ]:
BNEWAOI.drop(columns=['TARGET_FID', 'full_plus_', 'confidence', 'Shape_Area' , 'Shape_Leng' , 'Join_Count'], inplace=True)

## Preprocessing Buildings

In [ ]:
BNEWAOI.geometry = BNEWAOI.buffer(0)
BNEWAOI= BNEWAOI[~BNEWAOI.geometry.isna()]

In [ ]:
BNEWAOI.geom_type.value_counts()

In [ ]:
BNEWAOI = BNEWAOI.reset_index(drop=True).explode(index_parts=True).reset_index(drop=True)
BNEWAOI.geom_type.value_counts()

In [ ]:
BNEWAOI.columns

In [ ]:
duplicate_index = BNEWAOI.index.duplicated().any()
if duplicate_index:
    print("Some rows have the same index.")
else:
    print("All rows have unique indices.")

All rows have unique indices.


In [ ]:
BNEWAOI = BNEWAOI[BNEWAOI.area > 5]

In [ ]:
missing_height_count = BNEWAOI['HeightPROP'].isin([0, np.nan]).sum()
print(f"Number of missing height values: {missing_height_count}")

Number of missing height values: 22460


In [ ]:
rows_with_missing_height = BNEWAOI[BNEWAOI['HeightPROP'].isin([0, np.nan])]
print("\nRows with missing height values:")
print(rows_with_missing_height)

In [ ]:
mean_height = BNEWAOI['HeightPROP'].mean()
BNEWAOI.loc[BNEWAOI['HeightPROP'] == 0, 'HeightPROP'] = mean_height

In [ ]:
empty_height = BNEWAOI['HeightPROP'].isin([0, np.nan]).sum()
if empty_height:
    print("Null values still exist in the 'height' column.")
else:
    print("No null values in the 'height' column.")

No null values in the 'height' column.


In [ ]:
BNEWAOI["uID"] = range(len(BNEWAOI))
BNEWAOI['Height'] = BNEWAOI['HeightPROP'].round(2)
BNEWAOI.drop(columns=['HeightPROP'], inplace=True)
BNEWAOI.head()

In [ ]:
BNEWAOI.to_parquet('CleanBlgALL.pq')

## Generate tessellation

In [ ]:
Building = BNEWAOI

In [ ]:
checkVBuilding = mm.CheckTessellationInput( Building, shrink = 0.6)
checkV3 = mm.CheckTessellationInput(BNEWAOI_Tessinput, shrink = 0.6)

In [ ]:
BUILD = Building.drop(checkVBuilding.overlap.index.union(checkVBuilding.split.index))

In [ ]:
from shapely.geometry import MultiPolygon
multi_polygon_uids = []
for index, row in BUILD.iterrows():
    if isinstance(row.geometry, MultiPolygon):
        multi_polygon_uids.append(row['uID'])
print("UIDs of MultiPolygons:", multi_polygon_uids)

UIDs of MultiPolygons: []


In [ ]:
BUILD["uID"] = range(len(BUILD))
BUILD.head()

In [ ]:
limit = mm.buffered_limit(BUILD, 100)
%time TessALL = mm.Tessellation(BUILD, "uID", limit, segment = 0.7 , shrink = 0.6).tessellation

In [ ]:
BUILDS.to_parquet('CleanBlgNAOI.pq')
BUILDS.to_file('CleanBlgNAOI.shp')
TessALL.to_file('TessALL.shp')
TessALL.to_parquet('TessALL.pq')